# Supercomp - Atividade 11

In [1]:
!nvidia-smi

Mon Oct  7 17:30:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

- Cálculo da Variância usando Iteradores Dinâmicos
- Análise de Variação Diária dos Preços de Ações
- Contagem de Dias com Aumento no Preço das Ações
- Cálculo do Aumento Médio nos Dias em que o Preço Subiu

In [2]:
%%writefile atividade11.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/iterator/constant_iterator.h>
#include <thrust/extrema.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <thrust/count.h>
#include <iostream>
#include <chrono>
using namespace std;
using namespace std::chrono;

int main() {
    int n = 3112;
    double value = 0.0;
    thrust::host_vector<double> host(n, 0);
    for (int i = 0; i < n; i++) {
        cin >> value;
        host[i] = value;
    }
    thrust::device_vector<double> dev(host);


    // CALCULO DA VARIANCIA USANDO ITERADORES DINAMICOS

    auto inicio_leitura = chrono::steady_clock::now();
    double total_sum = thrust::reduce(dev.begin(), dev.end(), 0.0, thrust::plus<double>());
    double mean_price = total_sum / n;
    thrust::constant_iterator<double> mean_iter(mean_price);
    auto variance_functor = [mean_price] __host__ __device__ (double x) { return (x-mean_price) * (x-mean_price); };
    double variance = thrust::transform_reduce( dev.begin(), dev.end(), variance_functor, 0.0, thrust::plus<double>() ) / n;
    cout << "Variância dos preços das ações nos últimos 10 anos: " << variance << endl;
    auto fim_leitura = chrono::steady_clock::now();
    duration<double> diff = fim_leitura - inicio_leitura;
    cout << "Tempo do cálculo da variância (em segundos):  " << diff.count() << endl << endl;

    // ANALISE DA VARIACAO DIARIA DOS PRECOS DAS ACOES

    inicio_leitura = chrono::steady_clock::now();
    thrust::device_vector<double> ganho_diario(n-1);
    auto difference_functor = [] __host__ __device__ (double x1, double x2) { return x1 - x2; };
    thrust::transform(dev.begin(), dev.end()-1, dev.begin()+1, ganho_diario.begin(), difference_functor);
    cout << "Mostrando o ganho diário dos primeiros 20 dias:" << endl;
    for (int i = 0; i < 20; i++) { cout << "Ganho diário: " << ganho_diario[i] << endl; }
    fim_leitura = chrono::steady_clock::now();
    diff = fim_leitura - inicio_leitura;
    cout << "Tempo da análise da variação diária dos preços (em segundos):  " << diff.count() << endl << endl;

    // CONTAGEM DE DIAS COM AUMENTO NO PRECO DAS ACOES

    inicio_leitura = chrono::steady_clock::now();
    int dias_ganho = thrust::count_if(ganho_diario.begin(), ganho_diario.end(), [] __host__ __device__ (double x) { return x > 0; });
    cout << "Contagem de dias com ganho: " << dias_ganho << endl;
    fim_leitura = chrono::steady_clock::now();
    diff = fim_leitura - inicio_leitura;
    cout << "Tempo do cálculo da contagem de dias com aumento no preço (em segundos):  " << diff.count() << endl << endl;

    // CALCULO DO AUMENTO MEDIO NOS DIAS EM QUE O PRECO SUBIU

    inicio_leitura = chrono::steady_clock::now();
    thrust::replace_if(ganho_diario.begin(), ganho_diario.end(), [] __host__ __device__ (double x) { return x < 0; }, 0);
    double ganho_total = thrust::reduce(ganho_diario.begin(), ganho_diario.end(), 0.0, thrust::plus<double>());
    double aumento_medio = ganho_total / dias_ganho;
    cout << "Aumento médio nos dias que o preço subiu: " << aumento_medio << endl;
    fim_leitura = chrono::steady_clock::now();
    diff = fim_leitura - inicio_leitura;
    cout << "Tempo do cálculo do aumento médio nos dias que o preço subiu (em segundos):  " << diff.count() << endl;

    return 0;
}

Overwriting atividade11.cu


In [3]:
!nvcc -arch=sm_75 -std=c++14 --extended-lambda atividade11.cu -o atividade11

In [4]:
!./atividade11 < stocks-google.txt

Variância dos preços das ações nos últimos 10 anos: 123792
Tempo do cálculo da variância (em segundos):  0.00105211

Mostrando o ganho diário dos primeiros 20 dias:
Ganho diário: -5.26
Ganho diário: -1.5
Ganho diário: 3.25
Ganho diário: -6.25
Ganho diário: -3.5
Ganho diário: -3.25
Ganho diário: 1.75
Ganho diário: -5
Ganho diário: 2.5
Ganho diário: -5.5
Ganho diário: 5.25
Ganho diário: -6.25
Ganho diário: -1.5
Ganho diário: -2.5
Ganho diário: -2.25
Ganho diário: 5.25
Ganho diário: 1.5
Ganho diário: -3.75
Ganho diário: -5.5
Ganho diário: -1.25
Tempo da análise da variação diária dos preços (em segundos):  0.000448037

Contagem de dias com ganho: 55
Tempo do cálculo da contagem de dias com aumento no preço (em segundos):  0.000135606

Aumento médio nos dias que o preço subiu: 153.624
Tempo do cálculo do aumento médio nos dias que o preço subiu (em segundos):  8.7826e-05


## Discussão de Resultados

O tempo total de execução já é muito pequeno, mostrando que com grandes volumes de dados (como 10 anos de preços diários), essa abordagem pode escalar bem, mantendo o uso de memória controlado e o tempo de processamento eficiente.